# Projet de DataMining :  

Dans ce projet nous allons recommander des images en fonctions d'images déjà "liké".
Pour cela nous avons commencé par traiter une base de 733 images en recupérant le maximun de données. On recupére notament les données Exif, la taille de l'image, son orientation, ect ...  
La base de données d'images vient du lien suivant:
https://www.kaggle.com/aksha05/flower-image-dataset
## Requirements : 

In [ ]:
from PIL import Image
from pandas import json_normalize
import pandas as pd
import json
import os
import numpy
import math
import matplotlib.pyplot as plot
import time
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans

## Récupération des données et organisation de ces données dans un fichier json:

In [ ]:
path = "images/flowers"
dataJson = open('data.json', 'w')
dataTot = []
for file in os.listdir(path): #On peut lire l'ensemble des fichiers présents dans le dossier donné par path
    data = {}
    if file[0] != ".": #On exclue les fichiers "cachés"
        new_path = path + "/" + file
        image = Image.open(new_path) #Récupération de l'image avec le module Image de PIL
        tupleSize = image.size
        data["id_picture"] = int(file[-6:-4])
        data["name"] = file[:-10]
        data["size"] = tupleSize
        data["format"] = image.format
        if tupleSize[0] > tupleSize[1] : 
            data["orientation"] = "paysage"
        elif tupleSize[0] < tupleSize[1] :
            data["orientation"] = "portrait"
        else:
            data["orientation"] = "carre"
        dataTot.append(data)

json.dump(dataTot,dataJson, indent = 4)
dataJson.close()
dataTot_frame = json_normalize(dataTot)
print(dataTot_frame)



## Détection des couleurs prédominantes

In [ ]:
start_time = time.time()
imgfile = Image.open("images/flowers/daisies_00046.jpg")
n_clusters = 4 #int(input("Nombre de clusters pour l'algorithme des K-Means: "))
numarray = numpy.array(imgfile.getdata(), numpy.uint8)
clusters = KMeans(n_clusters)
clusters.fit(numarray)
npbins = numpy.arange(0, n_clusters+1)
histogram = numpy.histogram(clusters.labels_, bins=npbins)
labels = numpy.unique(clusters.labels_)
index1 = {}
for i,element in enumerate(histogram[0]):
    index1[element] = i
#print(index1)
histoSort = sorted(histogram[0], reverse=True)
#print(histoSort)
index2 = {}
for i,element in enumerate(histoSort):
    index2[i] = element
#print(index2)
barlist = plot.bar(labels, histoSort)
colors = []
for k in range(n_clusters):
    i = index1[index2[k]]
    colors.append('#%02x%02x%02x' % (
    math.ceil(clusters.cluster_centers_[i][0]), 
        math.ceil(clusters.cluster_centers_[i][1]),
    math.ceil(clusters.cluster_centers_[i][2])))
    barlist[k].set_color('#%02x%02x%02x' % (
    math.ceil(clusters.cluster_centers_[i][0]), 
        math.ceil(clusters.cluster_centers_[i][1]),
    math.ceil(clusters.cluster_centers_[i][2])))
plot.show()
end_time = time.time()
total_time = end_time - start_time
print(total_time)

In [ ]:
start_time = time.time()
imgfile = Image.open("images/flowers/daisies_00046.jpg")
n_clusters = 4
numarray = numpy.array(imgfile.getdata(), numpy.uint8)
clusters = MiniBatchKMeans(n_clusters)
clusters.fit(numarray)
npbins = numpy.arange(0, n_clusters+1)
histogram = numpy.histogram(clusters.labels_, bins=npbins)
labels = numpy.unique(clusters.labels_)
index1 = {}
for i,element in enumerate(histogram[0]):
    index1[element] = i
#print(index1)
histoSort = sorted(histogram[0], reverse=True)
#print(histoSort)
index2 = {}
for i,element in enumerate(histoSort):
    index2[i] = element
#print(index2)
barlist = plot.bar(labels, histoSort)
colors = []
for k in range(n_clusters):
    i = index1[index2[k]]
    colors.append('#%02x%02x%02x' % (
    math.ceil(clusters.cluster_centers_[i][0]), #RGB normalement 0:R 1:G 2:B
        math.ceil(clusters.cluster_centers_[i][1]),
    math.ceil(clusters.cluster_centers_[i][2])))
    barlist[k].set_color('#%02x%02x%02x' % (
    math.ceil(clusters.cluster_centers_[i][0]), 
        math.ceil(clusters.cluster_centers_[i][1]),
    math.ceil(clusters.cluster_centers_[i][2])))
plot.show()
end_time = time.time()
total_time = end_time - start_time
print(total_time)

Ici on remarque bien que l'algorithme MiniBatchKMeans va beaucoup plus vite que l'algorithme des KMeans bien qu'il soit moins précis. Nous décidons de l'utiliser tout de même car nous ne pouvons pas nous permettre de prendre trop de temps pour traiter les 700 images !